**Installing and importing libraries**




In [ ]:
%pip install hazm


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 316 kB 15.0 MB/s 
     |████████████████████████████████| 233 kB 63.0 MB/s 
     |████████████████████████████████| 1.4 MB 8.3 MB/s 
  Created wheel for nltk: filename=nltk-3.3-py3-none-any.whl size=1394486 sha256=b26b1748fd86e0912082d782f4e0e9da9d51e97f216da26c32efb1a546a67c67
  Stored in directory: /root/.cache/pip/wheels/9b/fd/0c/d92302c876e5de87ebd7fc0979d82edb93e2d8d768bf71fac4
  Created wheel for libwapiti: filename=libwapiti-0.2.1-cp37-cp37m-linux_x86_64.whl size=154918 sha256=e7aff0113c20bbc0799120a4d9e5a8cbaab753fd7c738e521c17de17722217cf
  Stored in directory: /root/.cache/pip/wheels/ab/b2/5b/0fe4b8f5c0e65341e8ea7bb3f4a6ebabfe8b1ac31322392dbf
Successfully built nltk libwapiti
  Attempting uninstall: nltk
    Found existing installation: nltk 3.7
    Uninstalling nltk-3.7:
      Successfully uninstalled nltk-3.7


In [ ]:
from __future__ import unicode_literals
from hazm import *
import random
import numpy as np
import tqdm
import codecs
import re

Initializing

In [ ]:
def process(text):
  normalizer = Normalizer()
  normalized = normalizer.normalize(text)
  tokens = word_tokenize(normalized)
  tokens_lem = [get_lemma_set(t) for t in tqdm.tqdm(tokens)]

  return tokens_lem,tokens

In [ ]:
def get_lemma_set(tok):
    lemmatizer = Lemmatizer()
    return lemmatizer.lemmatize(tok)

In [ ]:
shenase_mazi = {
    'م': 'اول شخص مفرد',
    'ی': 'دوم شخص مفرد',
    '': 'سوم شخص مفرد',
    'ست': 'سوم شخص مفرد',
    'یم': 'اول شخص جمع',
    'ید': 'دوم شخص جمع',
    'ند': 'سوم شخص جمع',
}
shenase_mozare = {
    'م': 'اول شخص مفرد',
    'ی': 'دوم شخص مفرد',
    'د': 'سوم شخص مفرد',
    'یم': 'اول شخص جمع',
    'ید': 'دوم شخص جمع',
    'ند': 'سوم شخص جمع',
}

s_mozare1 = r"(د|ند|ید|یم|ی|م)"
mazi_sade_re= r"\b({past_stem})({s})\b"
mazi_sade = f"{mazi_sade_re}"
mazi_estemrary_re= r"\b[می|نمی].*({past_stem})({s})\b"
mazi_estemrary = f"{mazi_estemrary_re}"
# mazi_mostamar_re= r"\b[داشت]({s})[می].*({past_stem})({s})\b"
mazi_mostamar_re= r"\b(داشت{s}).*[می|نمی].*({past_stem})({s})\b"
mazi_mostamar = f"{mazi_mostamar_re}"
mazi_baeid_re= r"\b({past_stem})[ه].*[بود]({s})\b"
mazi_baeid = f"{mazi_baeid_re}"
mazi_naghli_re= r"\b({past_stem})[ه].*[ا]({s})\b"
mazi_naghli = f"{mazi_naghli_re}"
hal_ekhbari_re= r"\b[می|نمی].*({present_stem})({s})\b"
hal_ekhbari = f"{hal_ekhbari_re}"
hal_eltazemi_re= r"\b[ب].*({present_stem})({s})\b"
hal_eltazemi = f"{hal_eltazemi_re}"
hal_mostamar_re= r"\b(دار{s}).*[می|نمی].*({present_stem})({s})\b"
hal_mostamar = f"{hal_mostamar_re}"
aiande_re = r"\b(خواه{s}).*({past_stem})\b"
aiande = f"{aiande_re}"

**The main Process**

In [ ]:
text = "کیلومتر شمار نمی شمارد"
verbs_details = []
tokens_lem , tokens = process(text)
# print(tokens_lem)
# print('\n', tokens)
verbs_lem = [(i, x) for (i, x) in enumerate(tokens_lem) if (re.search('.#.', x) != None)]

counted = False
verbs = []
for j in range(len(verbs_lem)):
    i, verb_lem = verbs_lem[j]
    if re.match('داشت|دار', verb_lem) != None and (j+1) < len(verbs_lem) and ((verbs_lem[j+1][0]) == i+1):
      current_verb = tokens[i] + " " + tokens[i+1]
      counted = True
      verbs.append((current_verb, verbs_lem[j+1][1]))
    elif not counted:
      current_verb = tokens[i]
      verbs.append((current_verb, verb_lem))
    else:
      counted = False

for current_verb, x in verbs:
    details = {}
    print(current_verb)
    print(x)
    if re.search('.#.', x) != None:
        past_stem, present_stem = x.split('#')
        if re.search(f'.?{past_stem}.?', current_verb):
          if re.search('خواه', current_verb):
            details['زمان'] = 'آینده'
            details['بن فعل'] = past_stem
            details['شخص'] = ""
            details['نوع'] = ""
            for s in shenase_mozare:
              aiande_format = aiande.format(past_stem = details['بن فعل'], s=s)
              if (re.match(aiande_format,current_verb)):
                details['نوع'] = "آینده ساده"
                details['شخص'] = shenase_mozare.get(s)
          else:
            details['زمان'] = 'گذشته'
            details['بن فعل'] = past_stem

            for s in shenase_mazi:
              mazi_estemrary_format = mazi_estemrary.format(past_stem = details['بن فعل'], s=s)
              mazi_mostamar_format = mazi_mostamar.format(past_stem = details['بن فعل'], s=s)
              mazi_sade_format = mazi_sade.format(past_stem = details['بن فعل'], s=s)
              mazi_baeid_format = mazi_baeid.format(past_stem = details['بن فعل'], s=s)
              mazi_naghli_format = mazi_naghli.format(past_stem = details['بن فعل'], s=s)
              if (re.match(mazi_estemrary_format, current_verb)):
                details['نوع'] = "گذشته استمراری"
                details['شخص'] = shenase_mazi.get(s)
              elif (re.match(mazi_mostamar_format, current_verb)):
                details['نوع'] = "گذشته مستمر"
                details['شخص'] = shenase_mazi.get(s)
              elif (re.match(mazi_sade_format, current_verb)):
                details['نوع'] = "گذشته ساده"
                details['شخص'] = shenase_mazi.get(s)
              elif (re.match(mazi_baeid_format, current_verb)):
                details['نوع'] = "گذشته بعید"
                details['شخص'] = shenase_mazi.get(s)
              elif (re.match(mazi_naghli_format, current_verb)):
                details['نوع'] = "گذشته نقلی"
                details['شخص'] = shenase_mazi.get(s)

            # remaining = current_verb.split(f'{past_stem}')
        elif re.search(f'.?{present_stem}.?', current_verb):
            details['زمان'] = 'حال'
            details['بن فعل'] = present_stem
            details['شخص'] = ""
            details['نوع'] = ""

            for s in shenase_mozare:
              hal_ekhbari_format = hal_ekhbari.format(present_stem = details['بن فعل'], s=s)
              hal_eltazemi_format = hal_eltazemi.format(present_stem = details['بن فعل'], s=s)
              hal_mostamar_format = hal_mostamar.format(present_stem = details['بن فعل'], s=s)

              if (re.match(hal_ekhbari_format,current_verb)):
                details['شخص'] = shenase_mozare.get(s)
                details['نوع'] = "حال اخباری"
              elif (re.match(hal_eltazemi_format,current_verb)):
                details['شخص'] = shenase_mozare.get(s)
                details['نوع'] = "حال التزامی"
              elif (re.match(hal_mostamar_format,current_verb)):
                details['شخص'] = shenase_mozare.get(s)
                details['نوع'] = "حال مستمر"

            # remaining = current_verb.split(f'{present_stem}')

        verbs_details.append(details)

for x in verbs_details:
    print(x)

100%|██████████| 3/3 [00:00<00:00,  3.79it/s]

نمی‌شمارد
شمرد#شمار
{'زمان': 'حال', 'بن فعل': 'شمار', 'شخص': 'سوم شخص مفرد', 'نوع': 'حال اخباری'}
